In [1]:
import time
import datetime
import math
import subprocess
import sys
import numpy as np
import re

In [2]:
import pandas as pd
import pickle
import matplotlib as plt
from IPython.display import display
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', -1)

C:\Users\lguerra5\AppData\Local\Continuum\anaconda3\envs\fnop\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


In [3]:
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.tokenize import word_tokenize

In [5]:
import textblob
from textblob import TextBlob

In [6]:
from string import punctuation

In [7]:
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')

In [30]:
ro_dtc_part_full = pd.read_csv('C:/Users/lguerra5/Documents/FNOP/DTC association - F150 case study/AWS_DTC_confirmed_subsystem_2017-21_F150_rev.csv')

In [33]:
ro_dtc_part_full.head()

vin  mdl_yr veh_line_cd trans_cd eng_cd  tis_wsd    milge  \
0  1FTEW1CF0HFA71072  2017    T/FC        T/C3     T/MF   23       26534.0   
1  1FTEW1CF3HKD86939  2017    T/FC        T/C3     T/MF   11       13098.0   
2  1FTEW1CG2HKC94715  2017    T/FC        T/EU     T/UR   46       82547.0   
3  1FTEW1CP0HKC02162  2017    T/FC        T/C3     T/WQ   27       20004.0   
4  1FTEW1CP4HKC03833  2017    T/FC        T/C3     T/WQ   25       55207.0   

  part_base     ro_date  \
0  15604     2018-11-17   
1  15604     2018-06-18   
2  12A650    2021-04-27   
3  10346     2019-06-17   
4  10346     2019-09-03   

                                                                                                                                                                                                                                                              cust_eng_txt  \
0  CUSTOMER STATES BATTERY WAS REPLACED A COUPLE MONTHS AGO BUT BATTERY KEEPS GIVING PROBLEM, CHECK STARTING SYSTEM, BATTERY ALTERNATOR AND STARTER, CHECK AND ADVISE,..., CUSTOMER STATES AT TIMES AFTER JUMP START CENTER CONSOLE IS BLANK OR BLACKED OUT, PLS CK ADVISE   
1  ENGINE WIL NOT CRANK OVER , SAEEMS TO HAVE BATTERY PO WER                                                                                                                                                                                                                 
2  CHECK ENGINE LIGHT                                                                                                                                                                                                                                                        
3  DRIVEABILITY-GAS CHARGE LIGHT ON                                                                                                                                                                                                                                          
4  CUST STATES BATTERY LIGHT COMES ON WHILE DRIVING FORD EXT WARR                                                                                                                                                                                                            

            ecu_dtc  
0  0x760_C101A_0x16  
1  0x730_U0553_0x00  
2  0x730_U0140_0x00  
3  0x726_B130F_0x14  
4  0x7E0_P0316_0x00

In [32]:
#strip whitespace from column
ro_dtc_part_full['part_base'] = ro_dtc_part_full['part_base'].str.strip()
ro_dtc_part_full['ecu_dtc'] = ro_dtc_part_full['ecu_dtc'].str.strip()

### Filter to select base part numbers only

In [34]:
# base part nos in charging and fuel subsystem only
bp_list = ['10346','15604','9F972','9G756','9F593','9H307','9D370','9F479','12A650','10C679','14A068']

In [35]:
ro_dtc_part_full = ro_dtc_part_full[ro_dtc_part_full['part_base'].isin(bp_list)]

In [36]:
ro_dtc_part_full

vin  mdl_yr veh_line_cd trans_cd eng_cd  tis_wsd  \
0       1FTEW1CF0HFA71072  2017    T/FC        T/C3     T/MF   23        
1       1FTEW1CF3HKD86939  2017    T/FC        T/C3     T/MF   11        
2       1FTEW1CG2HKC94715  2017    T/FC        T/EU     T/UR   46        
3       1FTEW1CP0HKC02162  2017    T/FC        T/C3     T/WQ   27        
4       1FTEW1CP4HKC03833  2017    T/FC        T/C3     T/WQ   25        
...                   ...   ...     ...         ...      ...   ..        
110474  1FTFX1E53MKD20164  2021    T/FD        T/EU     T/RAV  6         
110475  1FTFX1E55MKD57085  2021    T/FD        T/EU     T/RAV  2         
110476  1FTFX1E58MFB30823  2021    T/FD        T/EU     T/RAV  4         
110477  1FTMF1C51MKD56383  2021    T/FD        T/EU     T/RAV  4         
110478  1FTMF1C51MKD56383  2021    T/FD        T/EU     T/RAV  4         

          milge part_base     ro_date  \
0       26534.0  15604     2018-11-17   
1       13098.0  15604     2018-06-18   
2       82547.0  12A650    2021-04-27   
3       20004.0  10346     2019-06-17   
4       55207.0  10346     2019-09-03   
...         ...    ...            ...   
110474  8031.0   12A650    2021-07-20   
110475  3625.0   9F593     2021-05-13   
110476  2346.0   12A650    2021-07-07   
110477  5605.0   12A650    2021-07-07   
110478  5605.0   12A650    2021-07-07   

                                                                                                                                                                                                                                                                   cust_eng_txt  \
0       CUSTOMER STATES BATTERY WAS REPLACED A COUPLE MONTHS AGO BUT BATTERY KEEPS GIVING PROBLEM, CHECK STARTING SYSTEM, BATTERY ALTERNATOR AND STARTER, CHECK AND ADVISE,..., CUSTOMER STATES AT TIMES AFTER JUMP START CENTER CONSOLE IS BLANK OR BLACKED OUT, PLS CK ADVISE   
1       ENGINE WIL NOT CRANK OVER , SAEEMS TO HAVE BATTERY PO WER                                                                                                                                                                                                                 
2       CHECK ENGINE LIGHT                                                                                                                                                                                                                                                        
3       DRIVEABILITY-GAS CHARGE LIGHT ON                                                                                                                                                                                                                                          
4       CUST STATES BATTERY LIGHT COMES ON WHILE DRIVING FORD EXT WARR                                                                                                                                                                                                            
...                                                                ...                                                                                                                                                                                                            
110474  C/S HAVING PCM ISSUES                                                                                                                                                                                                                                                     
110475  CUSTOMER STATES THE CHECK ENGINE LIGHT IS ON CODE 0306 CYLINDER 6 MIR FIRE                                                                                                                                                                                                
110476  CUST ST: VEHICLE IS DEAD WILL NOT START. CHECK AND ADVISE                                                                                                    

In [37]:
#check number of unique base part numbers
ro_dtc_part_full.part_base.nunique()

11

In [38]:
#check number of unique DTC codes
ro_dtc_part_full.ecu_dtc.nunique()

1930

#### Save customer text column in separate df for processing

In [39]:
cust_txt = ro_dtc_part_full[['vin', 'ro_date', 'milge', 'cust_eng_txt']]

In [40]:
cust_txt['RO'] = cust_txt[['vin','ro_date']].apply(lambda x: ''.join(x), axis=1)

C:\Users\lguerra5\AppData\Local\Continuum\anaconda3\envs\fnop\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
# Drop cust text column for now
cust_txt.drop(columns = ['vin', 'ro_date'], inplace = True)

C:\Users\lguerra5\AppData\Local\Continuum\anaconda3\envs\fnop\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### Data quality checks

In [43]:
dq_check = cust_txt.groupby(['RO', 'milge'])['cust_eng_txt'].unique().reset_index()

In [44]:
dq_check

RO    milge  \
0      1FTEW1C40KKD107942019-05-14  354.0     
1      1FTEW1C40KKD446402020-02-10  2238.0    
2      1FTEW1C40KKD446402020-04-16  4940.0    
3      1FTEW1C40KKE468892020-06-12  7201.0    
4      1FTEW1C40LFA643072021-01-06  23010.0   
...                            ...      ...   
11231  1FTMF1EP8LKD901382021-03-30  16060.0   
11232  1FTMF1EP9KFA577452020-12-15  12831.0   
11233  1FTMF1EP9KKC734422021-03-18  8469.0    
11234  1FTMF1EP9KKD430052020-03-09  3637.0    
11235  1FTNF1EG2JKD036432019-06-07  31855.0   

                                                                                                                                      cust_eng_txt  
0      [CUSTOMER STATES ELECTRICAL CONCERN -CUSTOMERSTATES BACK UP CAMERA GIVES A MESSAGE FOR BACKUP CAMERA. BACK UP INOP, SCREEN BLANK ANDBLACK.]  
1      [KEY WILL NOT START TRUCK AT TIMES AND IF STARTEDWITH REMOTE WHEN TURNING KEY OR KEYS ON IN TRUCKIT WILL DIE]                                
2      [NO START WONT COME OUT OF PARK]                                                                                                             
3      [12V BATTERY TROUBLES / C/S MSG. CHECK CHARGING SYSTEM /]                                                                                    
4      [CUSTOMER STATES NOTHING TURNS ON WHEN YOU ENTER THE VEHICLE AND THE START BUTTON DOES NOT TURN ANYTHING ON- PLEASE CHECK AND ADVIS E]       
...                                                                                                                                       ...       
11231  [CUSTOMER STATES TOP LIGHT IN LEFT TURN SIGNAL INOP.]                                                                                        
11232  [CUSTOMER STATES TO HAVE CHECK ENGINE LIGHT ON]                                                                                              
11233  [CUSTOMER STATES CHECK ENGINE LIGHT, CHECK & ADVISE]                                                                                         
11234  [CUSTOMER STATES CHECK ENGINE LIGHT IS ON. RUNNING A LITTLE ROUGHER THAN NORMAL AT IDLE]                                                     
11235  [CUSTOMER STATES VEH WILL NOT START AT TIMES AND WHEN IT DOES START , CHECK ENGINE LIGHT IS ON & SEEMS TO IDLE ROUGH .]                      

[11236 rows x 3 columns]

In [45]:
len(dq_check)

11236

In [157]:
dq_check.cust_eng_txt[314]

array(['ADD ON LINE 6/12 SPW AFTER ROAD TEST CHARGING SYSTEM LIGHTCAME BACK ON',
       'ADD ON LINE AFTER ROAD TEST CHARGING SYSTEMLIGHT BACK ON',
       'CS CHARGING SYSTEM LIGHT ON'], dtype=object)

In [158]:
len(dq_check.cust_eng_txt[314])

3

In [151]:
count = 0
for i in range(len(dq_check)):
    if len(dq_check.cust_eng_txt[i]) > 2:
        print(i, ": ", dq_check.RO[i], ": ", dq_check.cust_eng_txt[i])
        count += 1
print(count)

314 :  1FTEW1C5XLKE241512020-06-01 :  ['ADD ON LINE 6/12 SPW AFTER ROAD TEST CHARGING SYSTEM LIGHTCAME BACK ON'
 'ADD ON LINE AFTER ROAD TEST CHARGING SYSTEMLIGHT BACK ON'
 'CS CHARGING SYSTEM LIGHT ON']
501 :  1FTEW1CGXHFB770702017-05-16 :  ['REPLACE PCM PER HOTLINE ASSISTANCE'
 'PER HOTLINE ASSISTANCE, REPLACE PCM'
 'CUSTOMER STATES THE CHECK ENGINE LIGHT AND WRENCHLIGHT ARE ILLUMINATEDIN THE INSTURMENT CLUSTER. WHEN DRIVING THE VEHICLE A LACK OF POWER I S EXPERIENCED']
704 :  1FTEW1CP3JKD784352019-11-19 :  ['CUSTOMER STATES CUSTOMER STATE`S AUTO STOP/START NOT WORKING PROPERLY'
 'CUSTOMER STATES CUSTOMER STATES WORKED AFTER BATTERY WAS RESET, NOW RARELY WORKS'
 'CUSTOMER STATES CUSTOMER STATE`S USUALLY STOPS AND RESTARTS SECONDS AFTER STOPPED, ALSO HAS A ROUGH START']
4528 :  1FTEW1EG6JFD673992020-11-11 :  ['CUSTOMER STATES MULTIPLE WARNING IN DASH.'
 'ELECTRICAL CUST STATES REVERSE LIGHTS ARE ON PERMENTLY ON. PLEASE CHECK AND ADVISE.'
 'CUSTOMER STATES VEHICLE WILL NOT TURN OFF. ON

In [ ]:
# 166 total that have more than 1 comment - 
# 163 that have 2 comments
# 6 that have 3 comments

In [50]:
#check multiple mileage records for 1 RO
dq_check2 = cust_txt.groupby(['RO'])['milge'].unique().reset_index()

In [51]:
dq_check2

RO      milge
0      1FTEW1C40KKD107942019-05-14  [354.0]  
1      1FTEW1C40KKD446402020-02-10  [2238.0] 
2      1FTEW1C40KKD446402020-04-16  [4940.0] 
3      1FTEW1C40KKE468892020-06-12  [7201.0] 
4      1FTEW1C40LFA643072021-01-06  [23010.0]
...                            ...        ...
11231  1FTMF1EP8LKD901382021-03-30  [16060.0]
11232  1FTMF1EP9KFA577452020-12-15  [12831.0]
11233  1FTMF1EP9KKC734422021-03-18  [8469.0] 
11234  1FTMF1EP9KKD430052020-03-09  [3637.0] 
11235  1FTNF1EG2JKD036432019-06-07  [31855.0]

[11236 rows x 2 columns]

In [52]:
count = 0
for i in range(len(dq_check2)):
    if len(dq_check2.milge[i]) > 1:
        print(i, ": ", dq_check2.RO[i], ": ", dq_check2.milge[i])
        count += 1
print(count)

0


In [159]:
# Create Final Cust txt df to be processed 
cust_txt_fin = cust_txt.groupby(['RO'])['cust_eng_txt'].unique().apply(' '.join).reset_index()

In [160]:
cust_txt_fin

RO  \
0      1FTEW1C40KKD107942019-05-14   
1      1FTEW1C40KKD446402020-02-10   
2      1FTEW1C40KKD446402020-04-16   
3      1FTEW1C40KKE468892020-06-12   
4      1FTEW1C40LFA643072021-01-06   
...                            ...   
11231  1FTMF1EP8LKD901382021-03-30   
11232  1FTMF1EP9KFA577452020-12-15   
11233  1FTMF1EP9KKC734422021-03-18   
11234  1FTMF1EP9KKD430052020-03-09   
11235  1FTNF1EG2JKD036432019-06-07   

                                                                                                                                    cust_eng_txt  
0      CUSTOMER STATES ELECTRICAL CONCERN -CUSTOMERSTATES BACK UP CAMERA GIVES A MESSAGE FOR BACKUP CAMERA. BACK UP INOP, SCREEN BLANK ANDBLACK.  
1      KEY WILL NOT START TRUCK AT TIMES AND IF STARTEDWITH REMOTE WHEN TURNING KEY OR KEYS ON IN TRUCKIT WILL DIE                                
2      NO START WONT COME OUT OF PARK                                                                                                             
3      12V BATTERY TROUBLES / C/S MSG. CHECK CHARGING SYSTEM /                                                                                    
4      CUSTOMER STATES NOTHING TURNS ON WHEN YOU ENTER THE VEHICLE AND THE START BUTTON DOES NOT TURN ANYTHING ON- PLEASE CHECK AND ADVIS E       
...                                                                                                                                     ...       
11231  CUSTOMER STATES TOP LIGHT IN LEFT TURN SIGNAL INOP.                                                                                        
11232  CUSTOMER STATES TO HAVE CHECK ENGINE LIGHT ON                                                                                              
11233  CUSTOMER STATES CHECK ENGINE LIGHT, CHECK & ADVISE                                                                                         
11234  CUSTOMER STATES CHECK ENGINE LIGHT IS ON. RUNNING A LITTLE ROUGHER THAN NORMAL AT IDLE                                                     
11235  CUSTOMER STATES VEH WILL NOT START AT TIMES AND WHEN IT DOES START , CHECK ENGINE LIGHT IS ON & SEEMS TO IDLE ROUGH .                      

[11236 rows x 2 columns]

### Count part repair vs DTC part repair

In [60]:
ro_dtc_part_full.drop(columns = ['cust_eng_txt'], inplace = True)

In [61]:
part_repair_count = ro_dtc_part_full.groupby('part_base')['vin'].count().to_frame(name='repair_count').reset_index()

In [62]:
part_dtc_repair_count = ro_dtc_part_full.groupby(['part_base', 'ecu_dtc'])['vin'].count().to_frame(name='dtc_repair_count').reset_index()

In [63]:
part_dtc_repair_count.shape

(8239, 3)

In [64]:
part_dtc_repair_count.head()

part_base           ecu_dtc  dtc_repair_count
0  10346     0x6F0_U0140_0x87  1               
1  10346     0x6F0_U3003_0x16  3               
2  10346     0x6F1_B012E_0x08  1               
3  10346     0x6F1_P2EA7_0x00  1               
4  10346     0x706_B115E_0x15  1

#### merge together repair parts and parts that we can associate DTC

In [65]:
part_dtc_summary = part_dtc_repair_count.merge(part_repair_count, 'inner')

In [66]:
len(list(set(list(part_dtc_summary['part_base']))))

11

In [67]:
part_dtc_summary['normalized_repair_count']=part_dtc_summary['dtc_repair_count']/part_dtc_summary['repair_count']

In [68]:
part_dtc_summary.head()

part_base           ecu_dtc  dtc_repair_count  repair_count  \
0  10346     0x6F0_U0140_0x87  1                 13382          
1  10346     0x6F0_U3003_0x16  3                 13382          
2  10346     0x6F1_B012E_0x08  1                 13382          
3  10346     0x6F1_P2EA7_0x00  1                 13382          
4  10346     0x706_B115E_0x15  1                 13382          

   normalized_repair_count  
0  0.000075                 
1  0.000224                 
2  0.000075                 
3  0.000075                 
4  0.000075

### plot part over the overall repair count histogram

In [69]:
part_overall_repair_df = part_dtc_summary[['part_base', 'repair_count']].drop_duplicates()

In [70]:
part_overall_repair_df['repair_count'].describe()

count    11.000000   
mean     10043.545455
std      8174.077090 
min      1494.000000 
25%      3003.000000 
50%      8051.000000 
75%      15618.000000
max      23416.000000
Name: repair_count, dtype: float64

In [71]:
part_overall_repair_df.head(5)

part_base  repair_count
0     10346     13382       
872   10C679    1701        
1307  12A650    17854       
2549  14A068    5591        
3452  15604     23416

In [267]:
#ax = part_overall_repair_df.boxplot()

#### keep only part that has been repaired for more than 5 instances

In [73]:
major_part_overall_repair = part_overall_repair_df.loc[lambda df: df['repair_count'] >=5, ]

In [268]:
#ax = major_part_overall_repair.boxplot()

In [75]:
major_part_dtc_summary = part_dtc_summary.loc[lambda df: df['repair_count'] >=5, ]

In [76]:
major_part_dtc_summary.sort_values('repair_count', ascending=False).head(10)

part_base           ecu_dtc  dtc_repair_count  repair_count  \
4119  15604     0x760_C0034_0x65  2                 23416          
4166  15604     0x760_U0131_0x87  2                 23416          
4152  15604     0x760_C2006_0x78  2                 23416          
4153  15604     0x760_C2007_0x19  1                 23416          
4154  15604     0x760_C2008_0x13  1                 23416          
4155  15604     0x760_C2008_0x19  1                 23416          
4156  15604     0x760_P050F_0x00  17                23416          
4157  15604     0x760_P05D3_0x86  21                23416          
4158  15604     0x760_P05E6_0x92  7                 23416          
4159  15604     0x760_P166B_0x12  1                 23416          

      normalized_repair_count  
4119  0.000085                 
4166  0.000085                 
4152  0.000085                 
4153  0.000043                 
4154  0.000043                 
4155  0.000043                 
4156  0.000726                 
4157  0.000897                 
4158  0.000299                 
4159  0.000043

#### check dtc count summary

In [77]:
dtc_count_summary = major_part_dtc_summary.groupby('ecu_dtc')['dtc_repair_count'].sum().to_frame(name='dtc_sum').reset_index()

In [78]:
dtc_count_summary.sort_values('dtc_sum', ascending=False).head(10)

ecu_dtc  dtc_sum
1125  0x7A7_U0452_0x82  3038   
767   0x754_U0140_0x00  2935   
58    0x706_C1001_0x97  2855   
473   0x730_U0140_0x00  2697   
771   0x754_U0253_0x00  2409   
766   0x754_U0100_0x00  2386   
153   0x720_U0140_0x00  2232   
502   0x730_U3003_0x68  2166   
1322  0x7E0_P0096_0x00  1827   
1122  0x7A7_U0253_0x00  1764

#### normalize the dtc repair count (major summary)

In [79]:
major_part_dtc_summary['normalized_repair_count']=major_part_dtc_summary['dtc_repair_count']/major_part_dtc_summary['repair_count']

In [80]:
major_part_dtc_pivot = major_part_dtc_summary.pivot(index='ecu_dtc',columns='part_base',values='normalized_repair_count')

In [81]:
major_part_dtc_pivot_df = major_part_dtc_pivot.fillna(0.0)

In [82]:
major_part_dtc_pivot_df.shape

(1930, 11)

In [83]:
major_part_dtc_pivot_df.head(5)

part_base         10346  10C679  12A650    14A068  15604  9D370  9F479  9F593  \
ecu_dtc                                                                         
0x6F0_C0040_0x11  0.0    0.0     0.0     0.000179  0.0    0.0    0.0    0.0     
0x6F0_P0CE2_0x00  0.0    0.0     0.0     0.000179  0.0    0.0    0.0    0.0     
0x6F0_P2CFC_0x00  0.0    0.0     0.0     0.000179  0.0    0.0    0.0    0.0     
0x6F0_P2CFD_0x00  0.0    0.0     0.0     0.000179  0.0    0.0    0.0    0.0     
0x6F0_P2CFE_0x00  0.0    0.0     0.0     0.000179  0.0    0.0    0.0    0.0     

part_base         9F972  9G756  9H307  
ecu_dtc                                
0x6F0_C0040_0x11  0.0    0.0    0.0    
0x6F0_P0CE2_0x00  0.0    0.0    0.0    
0x6F0_P2CFC_0x00  0.0    0.0    0.0    
0x6F0_P2CFD_0x00  0.0    0.0    0.0    
0x6F0_P2CFE_0x00  0.0    0.0    0.0

#### Use repairs with DTC data to build classification to build model input DF

In [84]:
### inner join with major part list
major_ro_dtc_part_df = ro_dtc_part_full.merge(major_part_overall_repair[['part_base']], 'inner')

In [85]:
len(set(list(major_ro_dtc_part_df['part_base'])))

11

In [86]:
major_ro_dtc_part_df['event'] = 1.0

In [87]:
major_ro_dtc_part_df['index'] = major_ro_dtc_part_df[['vin','ro_date','part_base']].apply(lambda x: ''.join(x), axis=1)

In [88]:
major_ro_dtc_part_df.head(5)

vin  mdl_yr veh_line_cd trans_cd eng_cd  tis_wsd    milge  \
0  1FTEW1CF0HFA71072  2017    T/FC        T/C3     T/MF   23       26534.0   
1  1FTEW1CF3HKD86939  2017    T/FC        T/C3     T/MF   11       13098.0   
2  1FTEW1EF2HKD92695  2017    T/FC        T/C3     T/MF   29       33488.0   
3  1FTEW1EFXHFB77123  2017    T/FC        T/C3     T/MF   4        6271.0    
4  1FTEW1EG2HFA18024  2017    T/FC        T/EU     T/UR   2        1321.0    

  part_base     ro_date           ecu_dtc  event  \
0  15604     2018-11-17  0x760_C101A_0x16  1.0     
1  15604     2018-06-18  0x730_U0553_0x00  1.0     
2  15604     2020-01-31  0x7A7_U0253_0x00  1.0     
3  15604     2017-07-31  0x7E0_P06DA_0x00  1.0     
4  15604     2017-02-03  0x736_U0100_0x00  1.0     

                              index  
0  1FTEW1CF0HFA710722018-11-1715604  
1  1FTEW1CF3HKD869392018-06-1815604  
2  1FTEW1EF2HKD926952020-01-3115604  
3  1FTEW1EFXHFB771232017-07-3115604  
4  1FTEW1EG2HFA180242017-02-0315604

In [89]:
# check duplicates on index column
a = major_ro_dtc_part_df.duplicated(subset='index', keep='first').sum()
print(a)

98642


In [90]:
#major_ro_dtc_part_pivot = major_ro_dtc_part_df.pivot(index='index',columns='ecu_dtc',values='event').reset_index()
major_ro_dtc_part_pivot = major_ro_dtc_part_df.pivot_table(index='index',columns='ecu_dtc',values='event').reset_index()

In [91]:
major_ro_dtc_part_pivot.shape

(11837, 1931)

In [92]:
major_ro_dtc_part_pivot['RO'] = major_ro_dtc_part_pivot['index'].apply(lambda x: x[:27])
major_ro_dtc_part_pivot['part_number'] = major_ro_dtc_part_pivot['index'].apply(lambda x: x[27:])

In [93]:
major_ro_dtc_part_pivot.head(5)

ecu_dtc                              index  0x6F0_C0040_0x11  \
0        1FTEW1C40KKD107942019-05-1415604  NaN                 
1        1FTEW1C40KKD446402020-02-1012A650 NaN                 
2        1FTEW1C40KKD446402020-04-1615604  NaN                 
3        1FTEW1C40KKE468892020-06-1210346  NaN                 
4        1FTEW1C40LFA643072021-01-0615604  NaN                 

ecu_dtc  0x6F0_P0CE2_0x00  0x6F0_P2CFC_0x00  0x6F0_P2CFD_0x00  \
0       NaN               NaN               NaN                 
1       NaN               NaN               NaN                 
2       NaN               NaN               NaN                 
3       NaN               NaN               NaN                 
4       NaN               NaN               NaN                 

ecu_dtc  0x6F0_P2CFE_0x00  0x6F0_U0100_0x87  0x6F0_U0121_0x87  \
0       NaN               NaN               NaN                 
1       NaN               NaN               NaN                 
2       NaN               NaN               NaN                 
3       NaN               NaN               NaN                 
4       NaN               NaN               NaN                 

ecu_dtc  0x6F0_U0140_0x87  0x6F0_U1000_0x00  0x6F0_U2100_0x00  \
0       NaN               NaN               NaN                 
1       NaN               NaN               NaN                 
2       NaN               NaN               NaN                 
3       NaN               NaN               NaN                 
4       NaN               NaN               NaN                 

ecu_dtc  0x6F0_U3000_0x49  0x6F0_U3003_0x16  0x6F1_B012E_0x08  \
0       NaN               NaN               NaN                 
1       NaN               NaN               NaN                 
2       NaN               NaN               NaN                 
3       NaN               NaN               NaN                 
4       NaN               NaN               NaN                 

ecu_dtc  0x6F1_B131A_0x16  0x6F1_B131A_0x17  0x6F1_P2EA7_0x00  \
0       NaN               NaN               NaN                 
1       NaN               NaN               NaN                 
2       NaN               NaN               NaN                 
3       NaN               NaN               NaN                 
4       NaN               NaN               NaN                 

ecu_dtc  0x6F1_U0100_0x00  0x6F1_U0100_0x87  0x6F1_U0151_0x00  \
0       NaN               NaN               NaN                 
1       NaN               NaN               NaN                 
2       NaN               NaN               NaN                 
3       NaN               NaN               NaN                 
4       NaN               NaN               NaN                 

ecu_dtc  0x6F1_U0151_0x87  0x6F1_U0293_0x00  0x6F1_U0401_0x00  \
0       NaN               NaN               NaN                 
1       NaN               NaN               NaN                 
2       NaN               NaN               NaN                 
3       NaN               NaN               NaN                 
4       NaN               NaN               NaN                 

ecu_dtc  0x6F2_U2008_0x08  0x6F2_U2100_0x00  0x706_B115E_0x15  \
0       NaN               NaN               NaN                 
1       NaN               NaN               NaN                 
2       NaN               NaN               NaN                 
3       NaN               NaN               NaN                 
4       NaN               NaN               NaN                 

ecu_dtc  0x706_B115E_0x31  0x706_B120C_0x11  0x706_B120C_0x13  \
0       NaN               NaN               NaN                 
1       NaN               NaN               NaN                 
2       NaN               NaN               NaN                 
3       NaN               NaN               NaN                 
4       NaN               NaN                1.0                

ecu_dtc  0x706_B129C_0x08  0x706_B129D_0x08  0x706_B12BE_0x15  \
0       NaN               NaN  

In [94]:
model_input_df = major_ro_dtc_part_pivot.iloc[:, 1:].fillna(0.0)

In [95]:
model_input_df.shape

(11837, 1932)

In [96]:
model_input_df.head(5)

ecu_dtc  0x6F0_C0040_0x11  0x6F0_P0CE2_0x00  0x6F0_P2CFC_0x00  \
0        0.0               0.0               0.0                
1        0.0               0.0               0.0                
2        0.0               0.0               0.0                
3        0.0               0.0               0.0                
4        0.0               0.0               0.0                

ecu_dtc  0x6F0_P2CFD_0x00  0x6F0_P2CFE_0x00  0x6F0_U0100_0x87  \
0        0.0               0.0               0.0                
1        0.0               0.0               0.0                
2        0.0               0.0               0.0                
3        0.0               0.0               0.0                
4        0.0               0.0               0.0                

ecu_dtc  0x6F0_U0121_0x87  0x6F0_U0140_0x87  0x6F0_U1000_0x00  \
0        0.0               0.0               0.0                
1        0.0               0.0               0.0                
2        0.0               0.0               0.0                
3        0.0               0.0               0.0                
4        0.0               0.0               0.0                

ecu_dtc  0x6F0_U2100_0x00  0x6F0_U3000_0x49  0x6F0_U3003_0x16  \
0        0.0               0.0               0.0                
1        0.0               0.0               0.0                
2        0.0               0.0               0.0                
3        0.0               0.0               0.0                
4        0.0               0.0               0.0                

ecu_dtc  0x6F1_B012E_0x08  0x6F1_B131A_0x16  0x6F1_B131A_0x17  \
0        0.0               0.0               0.0                
1        0.0               0.0               0.0                
2        0.0               0.0               0.0                
3        0.0               0.0               0.0                
4        0.0               0.0               0.0                

ecu_dtc  0x6F1_P2EA7_0x00  0x6F1_U0100_0x00  0x6F1_U0100_0x87  \
0        0.0               0.0               0.0                
1        0.0               0.0               0.0                
2        0.0               0.0               0.0                
3        0.0               0.0               0.0                
4        0.0               0.0               0.0                

ecu_dtc  0x6F1_U0151_0x00  0x6F1_U0151_0x87  0x6F1_U0293_0x00  \
0        0.0               0.0               0.0                
1        0.0               0.0               0.0                
2        0.0               0.0               0.0                
3        0.0               0.0               0.0                
4        0.0               0.0               0.0                

ecu_dtc  0x6F1_U0401_0x00  0x6F2_U2008_0x08  0x6F2_U2100_0x00  \
0        0.0               0.0               0.0                
1        0.0               0.0               0.0                
2        0.0               0.0               0.0                
3        0.0               0.0               0.0                
4        0.0               0.0               0.0                

ecu_dtc  0x706_B115E_0x15  0x706_B115E_0x31  0x706_B120C_0x11  \
0        0.0               0.0               0.0                
1        0.0               0.0               0.0                
2        0.0               0.0               0.0                
3        0.0               0.0               0.0                
4        0.0               0.0               0.0                

ecu_dtc  0x706_B120C_0x13  0x706_B129C_0x08  0x706_B129D_0x08  \
0        0.0               0.0               0.0                
1        0.0               0.0               0.0                
2        0.0               0.0               0.0                
3        0.0               0.0               0.0                
4        1.0               0.0               0.0                

ecu_dtc  0x706_B12BE_0x15  0x706_B12BE_0x31  0x706_B12BF_0x15  \
0        0.0             

### Group By Unique DTC Combo and RO

we want the unique parts number grouped by RO, not total, one row should have uique DTC combo -- part numbers repaired for ONE RO

In [97]:
col_list = model_input_df.columns.values
col_list = col_list[:-1]
col_list = col_list.tolist()
#col_list

In [98]:
grouped = model_input_df.groupby(col_list)['part_number'].unique().reset_index()

In [99]:
grouped

0x6F0_C0040_0x11  0x6F0_P0CE2_0x00  0x6F0_P2CFC_0x00  0x6F0_P2CFD_0x00  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               0.0                
11233  0.0               0.0               1.0               0.0                
11234  0.0               1.0               0.0               0.0                
11235  1.0               0.0               0.0               1.0                

       0x6F0_P2CFE_0x00  0x6F0_U0100_0x87  0x6F0_U0121_0x87  0x6F0_U0140_0x87  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               1.0               1.0               0.0                
11232  1.0               0.0               0.0               0.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F0_U1000_0x00  0x6F0_U2100_0x00  0x6F0_U3000_0x49  0x6F0_U3003_0x16  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               1.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F1_B012E_0x08  0x6F1_B131A_0x16  0x6F1_B131A_0x17  0x6F1_P2EA7_0x00  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               0.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F1_U0100_0x00  0x6F1_U0100_0x87  0x6F1_U0151_0x00  0x6F1_U0151_0x87  \
0      0.0               0.0      

In [100]:
# Check DTC and part number for first row
d = [c for c in grouped.columns if grouped[c][0] == 1] 
print(d)
print(grouped['RO'][0])
print(grouped['part_number'][0])

['0x7E1_P160A_0x00']
1FTEW1E4XKFA523542020-05-04
['12A650']


In [101]:
# check first row in input data
major_ro_dtc_part_df[(major_ro_dtc_part_df['ecu_dtc'] == '0x7E1_P160A_0x00') & (major_ro_dtc_part_df['part_base'] == '12A650')]

vin  mdl_yr veh_line_cd trans_cd eng_cd  tis_wsd  \
27012  1FTEW1EG7HFB52303  2017    T/FC        T/EU     T/UR   22        
32627  1FTEW1EG0JFA56745  2018    T/FC        T/EU     T/UR   9         
36235  1FTEW1E47KFA73162  2019    T/FC        T/EU     T/UR   12        
36248  1FTEW1E4XKKE78312  2019    T/FC        T/EU     T/UR   12        
36279  1FTEW1E4XKFA52354  2019    T/FC        T/EU     T/UR   9         
40194  1FTFW1RG2LFC62781  2020    T/FC        T/EU     T/KZ   4         

         milge part_base     ro_date           ecu_dtc  event  \
27012  34177.0  12A650    2019-03-25  0x7E1_P160A_0x00  1.0     
32627  10843.0  12A650    2018-07-18  0x7E1_P160A_0x00  1.0     
36235  17744.0  12A650    2020-10-23  0x7E1_P160A_0x00  1.0     
36248  19769.0  12A650    2021-04-06  0x7E1_P160A_0x00  1.0     
36279  23089.0  12A650    2020-05-04  0x7E1_P160A_0x00  1.0     
40194  3161.0   12A650    2021-03-09  0x7E1_P160A_0x00  1.0     

                                   index  
27012  1FTEW1EG7HFB523032019-03-2512A650  
32627  1FTEW1EG0JFA567452018-07-1812A650  
36235  1FTEW1E47KFA731622020-10-2312A650  
36248  1FTEW1E4XKKE783122021-04-0612A650  
36279  1FTEW1E4XKFA523542020-05-0412A650  
40194  1FTFW1RG2LFC627812021-03-0912A650

### Create DF with Additional Features to Add to Model

In [102]:
# create column to join dfs on
major_ro_dtc_part_df['RO'] = major_ro_dtc_part_df['index'].apply(lambda x: x[:27])

In [103]:
# group by RO
#add_feats = major_ro_dtc_part_df.groupby('RO').agg({'mdl_yr': 'first', 'veh_line_cd': 'first', 'trans_cd': 'first', 'eng_cd': 'first', 'tis_wsd': 'first', 'milge': 'first', 'avg_weibull_failure_rate': 'first'}).reset_index()
add_feats = major_ro_dtc_part_df.groupby('RO').agg({'mdl_yr': 'first', 'veh_line_cd': 'first', 'trans_cd': 'first', 'eng_cd': 'first', 'tis_wsd': 'first', 'milge': 'first'}).reset_index()

In [104]:
# create one-hot encoding for engine, trans, model year, vehicle line vars
#df_X = add_feats[['mdl_yr','tis_wsd','milge','avg_weibull_failure_rate','trans_cd', 'veh_line_cd', 'eng_cd']]
df_X = add_feats[['mdl_yr','tis_wsd','milge','trans_cd', 'veh_line_cd', 'eng_cd']]

In [105]:
df_X['mdl_yr'] = df_X['mdl_yr'].astype('object')

C:\Users\lguerra5\AppData\Local\Continuum\anaconda3\envs\fnop\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [107]:
## convert to one-hot encoding
df_X_dummy = pd.get_dummies(df_X)

In [111]:
df_X_dummy['RO'] = add_feats['RO']

In [112]:
df_X_dummy.shape

(11236, 23)

#### Add Vehicle features to model input DF
Run this if Only adding vehicle features to model input

In [114]:
# join with model input df
model_input_df_new = grouped.merge(df_X_dummy, on = 'RO', how = 'inner')

In [115]:
#fix column oder
cols = list(model_input_df_new.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('tis_wsd')) 
cols.pop(cols.index('milge'))
#cols.pop(cols.index('avg_weibull_failure_rate'))
cols.pop(cols.index('RO')) 
cols.pop(cols.index('part_number')) 
#model_input_new = model_input_df_new[cols+['tis_wsd','milge','avg_weibull_failure_rate','RO','part_number']] #Create new dataframe with columns in the order you want
model_input_new = model_input_df_new[cols+['tis_wsd','milge','RO','part_number']] #Create new dataframe with columns in the order you want
model_input_new

0x6F0_C0040_0x11  0x6F0_P0CE2_0x00  0x6F0_P2CFC_0x00  0x6F0_P2CFD_0x00  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               0.0                
11233  0.0               0.0               1.0               0.0                
11234  0.0               1.0               0.0               0.0                
11235  1.0               0.0               0.0               1.0                

       0x6F0_P2CFE_0x00  0x6F0_U0100_0x87  0x6F0_U0121_0x87  0x6F0_U0140_0x87  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               1.0               1.0               0.0                
11232  1.0               0.0               0.0               0.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F0_U1000_0x00  0x6F0_U2100_0x00  0x6F0_U3000_0x49  0x6F0_U3003_0x16  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               1.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F1_B012E_0x08  0x6F1_B131A_0x16  0x6F1_B131A_0x17  0x6F1_P2EA7_0x00  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               0.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F1_U0100_0x00  0x6F1_U0100_0x87  0x6F1_U0151_0x00  0x6F1_U0151_0x87  \
0      0.0               0.0      

#### Add vehicle Features and Reliability Data to Model Input Matrix

Run this section if adding reliability data AND vehicle features to model

In [116]:
#read in reliability data file
rel_df = pd.read_csv('C:/Users/lguerra5/Documents/FNOP/DTC association - F150 case study/reliability_data_for_F150_subsystem.csv')

In [117]:
rel_df.head(5)

model_year part_number_base  mis  avg_weibull_failure_rate
0  2017        10346            1    0.000050                
1  2017        10346            2    0.000112                
2  2017        10346            3    0.000181                
3  2017        10346            4    0.000253                
4  2017        10346            5    0.000329

In [118]:
rel_df.columns = ['mdl_yr', 'part_number','tis_wsd', 'Avg_Weibull_Failure']

In [119]:
data_reliab_pivot = rel_df.pivot_table(index=['mdl_yr', 'tis_wsd'], columns='part_number', values=['Avg_Weibull_Failure'], aggfunc='first')     
rel_df_t = data_reliab_pivot.fillna(0.0)

In [120]:
rel_df_t.shape

(300, 11)

In [122]:
df1 = pd.merge(df_X, rel_df_t, how='left', on=['mdl_yr', 'tis_wsd'])

C:\Users\lguerra5\AppData\Local\Continuum\anaconda3\envs\fnop\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [123]:
## convert to one-hot encoding
df_X_dummy_1 = pd.get_dummies(df1)

In [329]:
df_X_dummy_1.shape

(11236, 34)

In [125]:
df_X_dummy_1['RO'] = add_feats['RO']

In [126]:
# join with model input df
model_input_all_feats = grouped.merge(df_X_dummy_1, on = 'RO', how = 'inner')

In [127]:
#fix column oder
cols = list(model_input_all_feats.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('tis_wsd')) 
cols.pop(cols.index('milge'))
cols.pop(cols.index('RO')) 
cols.pop(cols.index('part_number')) 
model_input_all_feats = model_input_all_feats[cols+['tis_wsd','milge','RO','part_number']] #Create new dataframe with columns in the order you want
model_input_all_feats = model_input_all_feats.fillna(0.0)
model_input_all_feats

C:\Users\lguerra5\AppData\Local\Continuum\anaconda3\envs\fnop\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


0x6F0_C0040_0x11  0x6F0_P0CE2_0x00  0x6F0_P2CFC_0x00  0x6F0_P2CFD_0x00  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               0.0                
11233  0.0               0.0               1.0               0.0                
11234  0.0               1.0               0.0               0.0                
11235  1.0               0.0               0.0               1.0                

       0x6F0_P2CFE_0x00  0x6F0_U0100_0x87  0x6F0_U0121_0x87  0x6F0_U0140_0x87  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               1.0               1.0               0.0                
11232  1.0               0.0               0.0               0.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F0_U1000_0x00  0x6F0_U2100_0x00  0x6F0_U3000_0x49  0x6F0_U3003_0x16  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               1.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F1_B012E_0x08  0x6F1_B131A_0x16  0x6F1_B131A_0x17  0x6F1_P2EA7_0x00  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               0.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F1_U0100_0x00  0x6F1_U0100_0x87  0x6F1_U0151_0x00  0x6F1_U0151_0x87  \
0      0.0               0.0      

#### Add TFIDF data to Model Input Matrix

In [252]:
lookup_table = 'C:/Users/lguerra5/Documents/FNOP/data/Misspellings_Correction_Synonyms_Lookup_Table_Master.xlsx'

In [253]:
syn_df=pd.read_excel(lookup_table, engine='openpyxl')

In [254]:
syn_df = syn_df[syn_df['Review'] != "D"]

In [255]:
syn_dict=dict(zip(syn_df.misspelled, syn_df.corrected))

In [256]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J":'a',
                       "N":'n',
                       "V":'v',
                       "R":'r'}
    words_and_tags = [(w, tag_dict.get(pos[0],'n')) for w, pos in sent.tags]
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [257]:
def TextPreprocess(text):
    # Avoid mixed data type errors and apply str on all rows

    text = str(text).lower()

    #Remove HTML Tags
    text = re.sub('<[^>]*>', '', text)
    r1 = re.compile('<.*?>')
    text = re.sub(r1, ' ', text)
    # Remove any token with numbers and/or special characters

    r2 = re.compile(r'^[0-9\W\_]+$')
    text = re.sub(r2, ' ', text)

    #Removing Punctuations
    text = re.sub(r'[?|!|\'|"|#]',r'',text)
    text = re.sub(r'[.|,|)|(|\|/]',r' ',text)        

    # Tokenize all text data using nltk package

    text = word_tokenize(text)

    # Remove stop words
    stop = stopwords.words('english')
    stop1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','c/s','c/states','c/stated','cu', 'cs','c-s', 'cus','tomer', 'stomer', 'tate', 'cust',  'cust states', 'advise','nana','NaN','customer','states','rr','nan','wont','ck','check','pls','please', 'client','owner']

    # Combine default nltk stop list and custom stop list to generate a deduplicated list
    english_stops = list(set(stop+stop1))

    text = [item for item in text if item not in english_stops]

    # Remove special characters occuring as prefixes
    text = [re.sub(r'^[\W\_]+','',str(i)) for i in text]
    # Remove special characters occuring as suffixes

    text = [re.sub(r'[\W\_]+$','',str(i)) for i in text]

    # Replace special characters with spaces when occuring within the token 
    #(i.e. not at the beginning or the end) only for tokens greater than 5 characters in length

    text =[re.sub(r'[\W\_]+', ' ', str(i)) if len(str(i)) > 5 else i for i in text]

    # Replace special characters with null when occuring within the token 
    #(i.e. not at the beginning or the end) only for tokens less than 6 characters in length

    text = [re.sub(r'[\W\_]+', '', str(i)) if len(str(i)) < 6 else i for i in text]
    # Remove stop words after data cleansing steps to remove any residual stop words

    text = [item for item in text if item not in english_stops]

    # remove any numbers after data cleansing steps to remove any residual numeric values

    r3 = re.compile(r'^[0-9\W\_]+$')
    text = [i for i in text if not r3.match(i)]

    #Perform synonym lookup to find matching correctly spelled tokens for misspelled tokens
    #spell=SpellChecker() 

    text = [syn_dict.get(str(i), str(i)) for i in text]

    # Convert the list to string before passing to lemmatization module
    text = ' '.join(text)

    # Perform Lemmatization on the cleaned up data before tfidf vectorization

    text = lemmatize_with_postag(text)

    return(text)

In [262]:
cust_txt_fin['process_txt']=cust_txt_fin['cust_eng_txt'].apply(lambda x: TextPreprocess(x))

In [265]:
### convert null values of customer_txt into ' '
cust_txt_fin.loc[cust_txt_fin['process_txt'].isnull()==True,'process_txt'] = ' '
cust_txt_fin.drop(columns = ['cust_eng_txt'], inplace = True)

KeyError: "['milge'] not found in axis"

In [266]:
cust_txt_fin

RO  \
0      1FTEW1C40KKD107942019-05-14   
1      1FTEW1C40KKD446402020-02-10   
2      1FTEW1C40KKD446402020-04-16   
3      1FTEW1C40KKE468892020-06-12   
4      1FTEW1C40LFA643072021-01-06   
...                            ...   
11231  1FTMF1EP8LKD901382021-03-30   
11232  1FTMF1EP9KFA577452020-12-15   
11233  1FTMF1EP9KKC734422021-03-18   
11234  1FTMF1EP9KKD430052020-03-09   
11235  1FTNF1EG2JKD036432019-06-07   

                                                                                                          process_txt  
0      electrical concern customer state back camera give message backup camera back inoperable screen blank andblack  
1      key start truck time startedwith remote turn key key truckit die                                                
2      start come park                                                                                                 
3      12v battery trouble msg charge system                                                                           
4      nothing turn enter vehicle start button turn anything advise                                                    
...                                                             ...                                                    
11231  top light leave turn signal inoperable                                                                          
11232  engine light                                                                                                    
11233  engine light                                                                                                    
11234  engine light run little rougher normal idle                                                                     
11235  vehicle start time start engine light seem idle rough                                                           

[11236 rows x 2 columns]

In [272]:
vectorizer = TfidfVectorizer(ngram_range=(2,3), max_features = 500)

In [269]:
#vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df = 0.5, min_df = 0.0005)

In [53]:
# set vectorizing parameters for TFIDF
#max_words_param = 20000
#max_df_param = 0.95
#min_df_param = 3
#stops_list = stopwords.words('english')
#mf = 800

In [54]:
#vectorizer = TfidfVectorizer(stop_words=stops_list, min_df=min_df_param, max_df=max_df_param, ngram_range=(1,3), max_features = mf)

In [275]:
tfidf_matrix = vectorizer.fit_transform(cust_txt_fin['process_txt'])

In [276]:
tfidf_matrix.shape

(11236, 500)

In [277]:
tfidf_dense = tfidf_matrix.todense()

In [278]:
tfidf_features = vectorizer.get_feature_names()

In [281]:
tfidf_df = pd.DataFrame(tfidf_dense, columns = tfidf_features)

In [282]:
tfidf_df.head(5)

ad vise  add line  adv ise  advance trac  advise se  alarm go  and advise  \
0  0.0      0.0       0.0      0.0           0.0        0.0       0.0          
1  0.0      0.0       0.0      0.0           0.0        0.0       0.0          
2  0.0      0.0       0.0      0.0           0.0        0.0       0.0          
3  0.0      0.0       0.0      0.0           0.0        0.0       0.0          
4  0.0      0.0       0.0      0.0           0.0        0.0       0.0          

   auto start  auto start stop  auto stop  auto stop start  battery charge  \
0  0.0         0.0              0.0        0.0              0.0              
1  0.0         0.0              0.0        0.0              0.0              
2  0.0         0.0              0.0        0.0              0.0              
3  0.0         0.0              0.0        0.0              0.0              
4  0.0         0.0              0.0        0.0              0.0              

   battery dead  battery go  battery light  battery light come  bed light  \
0  0.0           0.0         0.0            0.0                 0.0         
1  0.0           0.0         0.0            0.0                 0.0         
2  0.0           0.0         0.0            0.0                 0.0         
3  0.0           0.0         0.0            0.0                 0.0         
4  0.0           0.0         0.0            0.0                 0.0         

   blow cold  body electrical  brake light  cel come  cel light  cel run  \
0  0.0        0.0              0.0          0.0       0.0        0.0       
1  0.0        0.0              0.0          0.0       0.0        0.0       
2  0.0        0.0              0.0          0.0       0.0        0.0       
3  0.0        0.0              0.0          0.0       0.0        0.0       
4  0.0        0.0              0.0          0.0       0.0        0.0       

   cel vehicle  charge battery  charge light  charge system  \
0  0.0          0.0             0.0           0.0             
1  0.0          0.0             0.0           0.0             
2  0.0          0.0             0.0           0.0             
3  0.0          0.0             0.0           1.0             
4  0.0          0.0             0.0           0.0             

   charge system fault  charge system light  charge system message  \
0  0.0                  0.0                  0.0                     
1  0.0                  0.0                  0.0                     
2  0.0                  0.0                  0.0                     
3  0.0                  0.0                  0.0                     
4  0.0                  0.0                  0.0                     

   charge system service  charge system warn  check and  check engine  \
0  0.0                    0.0                 0.0        0.0            
1  0.0                    0.0                 0.0        0.0            
2  0.0                    0.0                 0.0        0.0            
3  0.0                    0.0                 0.0        0.0            
4  0.0                    0.0                 0.0        0.0            

   check engine light  code p0096  cold start  collision assist  come back  \
0  0.0                 0.0         0.0         0.0               0.0         
1  0.0                 0.0         0.0         0.0               0.0         
2  0.0                 0.0         0.0         0.0               0.0         
3  0.0                 0.0         0.0         0.0               0.0         
4  0.0                 0.0         0.0         0.0               0.0         

   come dash  come drive  come engine  come exhaust  come go  come on  \
0  0.0        0.0         0.0          0.0           0.0      0.0       
1  0.0        0.0         0.0          0.0           0.0      0.0       
2  0.0        0.0         0.0          0.0           0.0      0.0       
3  0.0        0.0         0.0          0.0           0.0      0.0       
4  0.0        0.0         0.0          0.0           0.0    

#### combine tfidf_df with df_X_dummy_1 and merge with DTC to get final model input df

In [283]:
df_X_dummy_2 = pd.concat([df_X_dummy_1, tfidf_df], axis=1)

In [284]:
df_X_dummy_2.shape

(11236, 534)

In [286]:
# join with model input df
model_input_tfidf = grouped.merge(df_X_dummy_2, on = 'RO', how = 'inner')

In [287]:
#fix column oder
cols = list(model_input_tfidf.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('tis_wsd')) 
cols.pop(cols.index('milge'))
cols.pop(cols.index('RO')) 
cols.pop(cols.index('part_number')) 
model_input_tfidf = model_input_tfidf[cols+['tis_wsd','milge','RO','part_number']] #Create new dataframe with columns in the order you want
model_input_tfidf = model_input_tfidf.fillna(0.0)
model_input_tfidf

C:\Users\lguerra5\AppData\Local\Continuum\anaconda3\envs\fnop\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


0x6F0_C0040_0x11  0x6F0_P0CE2_0x00  0x6F0_P2CFC_0x00  0x6F0_P2CFD_0x00  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               0.0                
11233  0.0               0.0               1.0               0.0                
11234  0.0               1.0               0.0               0.0                
11235  1.0               0.0               0.0               1.0                

       0x6F0_P2CFE_0x00  0x6F0_U0100_0x87  0x6F0_U0121_0x87  0x6F0_U0140_0x87  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               1.0               1.0               0.0                
11232  1.0               0.0               0.0               0.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F0_U1000_0x00  0x6F0_U2100_0x00  0x6F0_U3000_0x49  0x6F0_U3003_0x16  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               1.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F1_B012E_0x08  0x6F1_B131A_0x16  0x6F1_B131A_0x17  0x6F1_P2EA7_0x00  \
0      0.0               0.0               0.0               0.0                
1      0.0               0.0               0.0               0.0                
2      0.0               0.0               0.0               0.0                
3      0.0               0.0               0.0               0.0                
4      0.0               0.0               0.0               0.0                
...    ...               ...               ...               ...                
11231  0.0               0.0               0.0               0.0                
11232  0.0               0.0               0.0               0.0                
11233  0.0               0.0               0.0               0.0                
11234  0.0               0.0               0.0               0.0                
11235  0.0               0.0               0.0               0.0                

       0x6F1_U0100_0x00  0x6F1_U0100_0x87  0x6F1_U0151_0x00  0x6F1_U0151_0x87  \
0      0.0               0.0      

### Use Multi-label Binarizer to encode multi part labels

In [129]:
#importing MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
#instantiating MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [130]:
#Encode the multilabel data in MLB Format
part_no_mlb = mlb.fit_transform(grouped['part_number'])
part_no_mlb

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [131]:
part_no_mlb.shape

(11236, 11)

In [132]:
#check first row
part_no_mlb[0]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [133]:
#mlb.inverse_transform(part_no_mlb)

In [134]:
mlb.classes_

array(['10346', '10C679', '12A650', '14A068', '15604', '9D370', '9F479',
       '9F593', '9F972', '9G756', '9H307'], dtype=object)

In [328]:
# Check model input matrix and size
#model_grouped.iloc[:,:-2] # DTC only
#model_input_new.iloc[:,:-2] # DTC and veh feats
#model_input_all_feats.iloc[:,:-2] # DTC + veh feats + rel
#model_input_tfidf.iloc[:,:-2] # DTC + veh feats + rel + cust txt

### Build Model using RF + Chain Classifier

In [137]:
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler, LabelEncoder
#from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin

In [138]:
from sklearn.metrics import accuracy_score, jaccard_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

In [139]:
from sklearn.multioutput import ClassifierChain

In [292]:
# Use inuput matrix with all features - DTC + veh + rel + TFIDF customer text
X_train, X_eval, y_train, y_eval = train_test_split(model_input_tfidf.iloc[:,:-2], part_no_mlb, test_size = 0.2, random_state = 11)

In [220]:
# Use inuput matrix with features - DTC + veh + rel
X_train, X_eval, y_train, y_eval = train_test_split(model_input_all_feats.iloc[:,:-2], part_no_mlb, test_size = 0.2, random_state = 11)

In [188]:
# Use inuput matrix with features - DTC + veh
X_train, X_eval, y_train, y_eval = train_test_split(model_input_new.iloc[:,:-2], part_no_mlb, test_size = 0.2, random_state = 11)

In [140]:
# use DTC only as input
X_train, X_eval, y_train, y_eval = train_test_split(grouped.iloc[:,:-2], part_no_mlb, test_size = 0.2, random_state = 11)

In [293]:
clf = RandomForestClassifier(n_estimators = 100, criterion = 'gini', min_samples_split = 3, min_samples_leaf = 1, max_features = 'auto')

In [294]:
chain = ClassifierChain(clf, order='random', random_state=0)

In [295]:
chain.fit(X_train, y_train)

ClassifierChain(base_estimator=RandomForestClassifier(min_samples_split=3),
                order='random', random_state=0)

In [296]:
y_pred1 = chain.predict(X_eval)

In [297]:
y_prob1 = chain.predict_proba(X_eval)

### Calculate Metrics - RF 

In [298]:
# exact match accuracy
accuracy_score(y_eval, y_pred1)

0.541814946619217

In [299]:
partwise_recall_chain = np.array([recall_score(y_eval[:, i], y_pred1[:, i]) for i in range(y_eval.shape[1])])

In [300]:
partwise_recall_chain

array([0.84070796, 0.20930233, 0.175     , 0.10606061, 0.55412371,
       0.02631579, 0.80962801, 0.70923379, 0.17322835, 0.        ,
       0.31413613])

In [301]:
partwise_precision_chain = np.array([precision_score(y_eval[:, i], y_pred1[:, i]) for i in range(y_eval.shape[1])])

C:\Users\lguerra5\AppData\Local\Continuum\anaconda3\envs\fnop\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [302]:
partwise_precision_chain

array([0.89201878, 0.75      , 0.77777778, 0.875     , 0.90336134,
       1.        , 0.95115681, 0.78649237, 0.62857143, 0.        ,
       0.85714286])

In [303]:
partwise_f1_chain = np.array([f1_score(y_eval[:, i], y_pred1[:, i]) for i in range(y_eval.shape[1])])

In [304]:
partwise_f1_chain

array([0.86560364, 0.32727273, 0.28571429, 0.18918919, 0.68690096,
       0.05128205, 0.87470449, 0.74586777, 0.27160494, 0.        ,
       0.45977011])

#### Aggregated Matrics - Mico and Macro - RF Chain

In [305]:
#### macro recall
## simple average of recall over all classes
recall_macro_chain = np.mean(partwise_recall_chain)

In [306]:
recall_macro_chain

0.3561578790609152

In [307]:
precision_macro_chain = np.mean(partwise_precision_chain)

In [308]:
precision_macro_chain

0.765592852221782

In [309]:
f1_macro_chain = np.mean(partwise_f1_chain)

In [310]:
f1_macro_chain

0.4325372881915069

In [311]:
#### micro recall
#### total number of TP / total number of True in y_true
recall_micro_chain = sum([sum(np.logical_and(y_eval[:,i], y_pred1[:,i])) for i in range(y_eval.shape[1])]) / sum([sum(y_eval[:,i]) for i in range(y_eval.shape[1])])

In [312]:
recall_micro_chain

0.5388166177087704

In [313]:
#### micro precision
#### total number of TP / total number of True in y_pred
precision_micro_chain = sum([sum(np.logical_and(y_eval[:,i], y_pred1[:,i])) for i in range(y_eval.shape[1])]) / sum([sum(y_pred1[:,i]) for i in range(y_eval.shape[1])])

In [314]:
precision_micro_chain

0.8629032258064516

In [315]:
f1_micro_chain = 2*recall_micro_chain*precision_micro_chain/(recall_micro_chain+precision_micro_chain)

In [316]:
f1_micro_chain

0.6633944717127357

#### Generate Metrics Report

In [317]:
### get number of parts in the evaluation set
num_part_eval = [sum(y_eval[:,i]) for i in range(y_eval.shape[1])]

In [318]:
num_part_eval

[226, 43, 280, 66, 388, 38, 457, 509, 127, 58, 191]

In [319]:
y_labels = mlb.classes_

In [320]:
report_df = pd.DataFrame(np.array([y_labels, num_part_eval, partwise_recall_chain, partwise_precision_chain, partwise_f1_chain]).T, columns=['target', 'target_count', 'recall', 'precision', 'f1-score'])

In [321]:
report_df = report_df.append({'target':'macro', 'target_count': '', 'recall':recall_macro_chain, 'precision': precision_macro_chain, 'f1-score':f1_macro_chain}, ignore_index=True)

In [322]:
report_df = report_df.append({'target':'micro', 'target_count': '', 'recall':recall_micro_chain, 'precision': precision_micro_chain, 'f1-score':f1_micro_chain}, ignore_index=True)

In [323]:
report_df

target target_count     recall precision   f1-score
0   10346   226          0.840708   0.892019  0.865604 
1   10C679  43           0.209302   0.75      0.327273 
2   12A650  280          0.175      0.777778  0.285714 
3   14A068  66           0.106061   0.875     0.189189 
4   15604   388          0.554124   0.903361  0.686901 
5   9D370   38           0.0263158  1         0.0512821
6   9F479   457          0.809628   0.951157  0.874704 
7   9F593   509          0.709234   0.786492  0.745868 
8   9F972   127          0.173228   0.628571  0.271605 
9   9G756   58           0          0         0        
10  9H307   191          0.314136   0.857143  0.45977  
11  macro                0.356158   0.765593  0.432537 
12  micro                0.538817   0.862903  0.663394